## SEARCH FOR DM SIGNALS FROM ELLIPTICAL GALAXIES - A STACKING ANALYSIS
A. Steinhebel, GSFC, 2022

#### FIRST STEP OF ANALYSIS PIPELINE: Consider catalog(s) of elliptical galaxies and remove those not suitable for analysis. Require that:
|b|>15deg ; >1deg separation between EG and blazar (sources from BZCAT catalog) ; >1 deg separation between EG and radio galaxy (sources from 2MRS catalog)

##### Import necessary packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import astropy
from astropy import units as u
from astropy.coordinates import SkyCoord
import math
from astropy.io import fits
from astropy.table import Table

##### Import and format EG catalog (https://arxiv.org/pdf/1304.7762.pdf) 

In [2]:
egDF=pd.read_csv('EGs_new.csv')

#get l,b from other file
egDF_mn=pd.read_csv('EGsTable.csv')
egDF_mn[['Name','Type','Distance[Mpc]','Distance Uncert','Ks','M_KsT','M_VT','(V-Ks)_0','(B-V)_0','M_BH[Msolar*10e6]','Flags:M','Flags:C','Flags:M_BH','l','b']]=egDF_mn['Name;Type;Distance[Mpc];Ks;M_KsT;M_VT;(V-Ks)_0;(B-V)_0;M_BH[Msolar*10e6];Flags:M;Flags:C;Flags:M_BH;;l;b'].str.split(';', expand=True)
egDF_mn.drop(columns=['Name;Type;Distance[Mpc];Ks;M_KsT;M_VT;(V-Ks)_0;(B-V)_0;M_BH[Msolar*10e6];Flags:M;Flags:C;Flags:M_BH;;l;b'], inplace=True)
egDF['l']=egDF_mn['l']
egDF['b']=egDF_mn['b']
#Convert l,b to floats
toConvert=['l','b']
for colu in toConvert:
    egDF[colu]=pd.to_numeric(egDF[colu])

In [3]:
#Remove rows of NaN entries
egDF.dropna(inplace=True)
totEGs=len(egDF.index)

In [4]:
#Add RA/Dec in degrees to egDF
c=SkyCoord(l = egDF['l']*u.deg, b = egDF['b']*u.deg, frame='galactic')
eg_ra=[]
eg_dec=[]
for coord in c.icrs: #convert galactic coordinates to icrs RA/Dec
    strcoord=coord.to_string()
    eg_ra.append(float(strcoord.split(' ')[0]))
    eg_dec.append(float(strcoord.split(' ')[1]))
egDF['RA']=eg_ra
egDF['Dec']=eg_dec

cl1 = c.icrs #egDF SkyCoord object

Save elements to remove in an array - at the end of the script, remove these elements (allows for simpler iteration)

In [5]:
toRemove=[]

In [6]:
#Remove candidates too close to galactic plane
removed=0
for x in egDF.index:
    if abs(egDF.loc[x,'b'])<15:
        toRemove.append(x) #remove row if b<15deg
        removed+=1
print(f"Removed {removed} EGs - left with {(totEGs-removed)/totEGs:.2f}% of sample")


Removed 2 EGs - left with 0.96% of sample


##### Compare to BZCAT Blazar catalog (https://www.ssdc.asi.it/bzcat5/) 


Define separation value to be considered overlap

In [7]:
sep=1.0 
max_sep = sep*u.degree #Maximum distance to neighbor

In [8]:
#Import catalog
blazarDF=pd.read_csv('bzcat_blazarCatalog.csv')
cl2 = SkyCoord(ra = blazarDF[' R.A. (J2000) ']*u.degree,dec = blazarDF[' Dec. (J2000) ']*u.degree)

In [9]:
#return arrays of elements from cl2 that are closest to element in cl1
#index of cl2 object closest to the corresponding cl1 object, 2D distance between objects, 3D distance between object (if exists)
idx, d2d, d3d = cl1.match_to_catalog_sky(cl2)

In [10]:
#Identify closest source and mark for removal if separation<sep
sep_constraint = d2d < max_sep #array of booleans corresponding to cl1 - True if sep too small and cl1 element should be removed
print(f"Eliminating {sep_constraint.sum()} EGs for overlap with BZCAT catalog")

for i in range(len(egDF)):
    if sep_constraint[i]:
        toRemove.append(i)

Eliminating 17 EGs for overlap with BZCAT catalog


##### Compare to 2MRS Radio galaxy catalog (http://ragolu.science.ru.nl/index.html))

In [11]:
#import catalog
radioDF=pd.read_csv('2mrs_radioCatalog.csv')
cl3 = SkyCoord(ra = radioDF['ra']*u.degree,dec = radioDF['dec']*u.degree)

In [12]:
#return array of elements from cl3 that are closest to element in cl1
idx, d2d, d3d = cl1.match_to_catalog_sky(cl3)

In [13]:
#Identify closest source and mark for removal if separation<sep
sep_constraint = d2d < max_sep #array of booleans corresponding to cl1 - True if sep too small and cl1 element should be removed
print(f"Eliminating {sep_constraint.sum()} EGs for overlap with 2MRS catalog - within {sep} deg")

for i in range(len(egDF)):
    if sep_constraint[i]:
        toRemove.append(i)

Eliminating 19 EGs for overlap with 2MRS catalog - within 1.0 deg


##### Remove all EGs marked for removal

In [14]:
toRemove=list(set(toRemove))
toRemove.sort()
for i in toRemove:
    egDF.drop(i,inplace=True)

print(f"Left with {len(egDF)} of the original {totEGs} EGs - {100*len(egDF)/totEGs:.2f}%")

Left with 17 of the original 45 EGs - 37.78%


##### Save resulting EG list as new csv

In [15]:
finalList="EGs_overlapRemoved.csv"
print(f"Saving the remaining EGs to {finalList}")
egDF.to_csv(finalList, index=False)

Saving the remaining EGs to EGs_overlapRemoved.csv
